

Hier sind k-fold cross-validation Scores für die Modelle Logistic Regression, Random Forest und K-Nearest Neighbors wobei die Daten gewichtet wurden. (tödliche bzw tödliche/ schwere Unfälle mit Gewichtung 500)

   - tödliche Unfälle vs. nicht tödliche Unfälle

alle Modelle haben eine geringe Precision, d.h. viele False Positives, d.h. viele fälschlicherweise als tod klassifiziert werden. Der F1 Score ist dementsprechend auch gering, d.h. das Modell hat Schwierigkeiten die tödlichen Unfälle zu erkennen.

   - tödliche und schwere Unfälle vs. leichte Unfälle

Der F1 Score ist etwas besser, d.h. das Modell hat weniger Schwierigkeiten die schweren oder tödlichen Unfälle zu erkennen, aber die Scores zeigen immer noch problematiche Precision und Recall Werte.

Nächste Schritte:

    - Überlegen, ob für den F1 Score Precision, Recall gewichtet werden soll? F1Beta-Score (Beta=1) = 2 * (Precision * Recall) / (Precision + Recall)
   -


In [4]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X=df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]
#für tödliche Unfälle
y=(df['UKATEGORIE'] == 1).astype(int)


# KFold-Konfiguration
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Logistische Regression
log_reg = LogisticRegression(max_iter=1000, class_weight={0:1, 1: 500})
accuracy_log = cross_val_score(log_reg, X, y, cv=kf, scoring='accuracy')
print("Accuracy Logistic Regression (k-fold):", accuracy_log.mean())


recall_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='recall')
print("Recall Logistic Regression  (k-fold):", recall_reg.mean())

precision_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='precision')
print("Precision Logistic Regression (k-fold):", precision_reg.mean())

f1_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='f1')
print("F1 Logistic Regression (k-fold):", f1_reg.mean())

roc_auc_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='roc_auc')
print("Roc Auc Logistic Regression (k-fold):", roc_auc_reg.mean())


# Random Forest
rf_clas = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42, class_weight={0:1, 1: 1000})
accuracy_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='accuracy')
print("Accuracy Random Forest (k-fold):", accuracy_rf.mean())

recall_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='recall')
print("Recall Random Forest (k-fold):", recall_rf.mean())

precision_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='precision')
print("Precision Random Forest (k-fold):", precision_rf.mean())

f1_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='f1')
print("F1 Random Forest (k-fold):", f1_rf.mean())

roc_auc_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='roc_auc')
print("Roc Auc Random Forest (k-fold):", roc_auc_rf.mean())


# K-Nearest Neighbors
knn_clas = KNeighborsClassifier(n_neighbors=3, weights='distance')
accuracy_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='accuracy')
print("Accuracy K-Nearest Neighbors (k-fold):", accuracy_knn.mean())

recall_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='recall')
print("Recall K-Nearest Neighbors (k-fold):", recall_knn.mean())

precision_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='precision')
print("Precision K-Nearest Neighbors (k-fold):", precision_knn.mean())

f1_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='f1')
print("F1 K-Nearest Neighbors (k-fold):", f1_knn.mean())

Accuracy Logistic Regression (k-fold): 0.7154711302870119
Recall Logistic Regression  (k-fold): 0.7700535945276478
Precision Logistic Regression (k-fold): 0.00825973665992595
F1 Logistic Regression (k-fold): 0.01633222148972382
Roc Auc Logistic Regression (k-fold): 0.8090594388855494
Accuracy Random Forest (k-fold): 0.7560354878910192
Recall Random Forest (k-fold): 0.6910267852764347
Precision Random Forest (k-fold): 0.008998802713895197
F1 Random Forest (k-fold): 0.017751517831859295
Roc Auc Random Forest (k-fold): 0.791631847415896
Accuracy K-Nearest Neighbors (k-fold): 0.9968474424227309
Recall K-Nearest Neighbors (k-fold): 0.0


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision K-Nearest Neighbors (k-fold): 0.0
F1 K-Nearest Neighbors (k-fold): 0.0




zB Logistische Regression:

    Recall: 76 % der tödlich Fälle wurden als tödlich erkannt
    Precision: 0,8 % der erkannten tödlich Fälle sind tatsächlich tödlich Fälle ( Precision = tp / (tp + fp) , d.h. wenn es viele False Positive gibt, viele fälschlicherweise als tod klassifiziert werden, dann wird der Nenner klein)



In [5]:
#für tödliche und schwere vs. leichte Unfälle
y = df['UKATEGORIE'].isin([1, 2]).astype(int)
# -> 1 ist schwer oder tödlich, 0 ist leicht

# KFold-Konfiguration
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# Logistische Regression
log_reg = LogisticRegression(max_iter=1000, class_weight={0:1, 1: 500})
accuracy_log = cross_val_score(log_reg, X, y, cv=kf, scoring='accuracy')
print("Accuracy Logistic Regression (k-fold):", accuracy_log.mean())


recall_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='recall')
print("Recall Logistic Regression  (k-fold):", recall_reg.mean())

precision_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='precision')
print("Precision Logistic Regression (k-fold):", precision_reg.mean())

f1_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='f1')
print("F1 Logistic Regression (k-fold):", f1_reg.mean())

roc_auc_reg = cross_val_score(log_reg, X, y, cv=kf, scoring='roc_auc')
print("Roc Auc Logistic Regression (k-fold):", roc_auc_reg.mean())


# Random Forest
rf_clas = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42, class_weight={0:1, 1: 100})
accuracy_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='accuracy')
print("Accuracy Random Forest (k-fold):", accuracy_rf.mean())

recall_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='recall')
print("Recall Random Forest (k-fold):", recall_rf.mean())

precision_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='precision')
print("Precision Random Forest (k-fold):", precision_rf.mean())

f1_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='f1')
print("F1 Random Forest (k-fold):", f1_rf.mean())

roc_auc_rf = cross_val_score(rf_clas, X, y, cv=kf, scoring='roc_auc')
print("Roc Auc Random Forest (k-fold):", roc_auc_rf.mean())


# K-Nearest Neighbors
knn_clas = KNeighborsClassifier(n_neighbors=3, weights='distance')
accuracy_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='accuracy')
print("Accuracy K-Nearest Neighbors (k-fold):", accuracy_knn.mean())

recall_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='recall')
print("Recall K-Nearest Neighbors (k-fold):", recall_knn.mean())

precision_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='precision')
print("Precision K-Nearest Neighbors (k-fold):", precision_knn.mean())

f1_knn = cross_val_score(knn_clas, X, y, cv=kf, scoring='f1')
print("F1 K-Nearest Neighbors (k-fold):", f1_knn.mean())

Accuracy Logistic Regression (k-fold): 0.1537371408091796
Recall Logistic Regression  (k-fold): 1.0
Precision Logistic Regression (k-fold): 0.1537371408091796
F1 Logistic Regression (k-fold): 0.26650076510709847
Roc Auc Logistic Regression (k-fold): 0.6400651281561294
Accuracy Random Forest (k-fold): 0.1537371408091796
Recall Random Forest (k-fold): 1.0
Precision Random Forest (k-fold): 0.1537371408091796
F1 Random Forest (k-fold): 0.26650076510709847
Roc Auc Random Forest (k-fold): 0.661834102835406
Accuracy K-Nearest Neighbors (k-fold): 0.8094496544619363
Recall K-Nearest Neighbors (k-fold): 0.11450133643685496
Precision K-Nearest Neighbors (k-fold): 0.24454579950201505
F1 K-Nearest Neighbors (k-fold): 0.15588427406712574




zB Logistische Regression:

    Recall: 100 % der schweren/tödlichen Fälle wurden als schweren/tödlichen erkannt
    Precision: 15 % der erkannten schweren/tödlichen Fälle sind tatsächlich schweren/tödlichen Fälle
    ROC AUC Logistic Regression: Der Wert von etwa 0,6393 zeigt, dass das Modell Logistic Regression eine mäßige Fähigkeit hat, zwischen den Klassen zu unterscheiden.

